In [1]:
from requests import get
from bs4 import BeautifulSoup as bs
import pandas as pd
import csv
from time import sleep

def get_last_page(url: str) -> str:
    pages = get(url)
    pages_lida = bs(pages.text, 'html.parser')
    links = pages_lida.find('span', {'class': 'wp-pager-pages'}).find_all('a')
    return max([link.get_text() for link in links])

def get_rank(url):
    #pagina = get(url)
    #ler_pg = bs(pagina.text, 'html.parser')
    rank_tag = ler_pg.find_all('td', {'class':'IndividualStatsRank'})
    ranks = [rank.get_text() for rank in rank_tag]
    return ranks

def get_player(url):
    #pagina = get(url)
    #ler_pg = bs(pagina.text, 'html.parser')
    player_tag = ler_pg.find_all('a', {'class':'StatsCenterPlayerName'})
    players = [nome.find('span', {'class':'hidden-xs'}).get_text() for nome in player_tag]
    return players

def get_team(url):
    #pagina = get(url)
    #ler_pg = bs(pagina.text, 'html.parser')
    team_tag = ler_pg.find_all('td', {'class':'StatsCenterTeams'})
    team = []
    for n in team_tag:
        if n.find('span', {'class':'visible-sm-inline'}) != None:
            team.append(n.find('span', {'class':'visible-md-inline'}).get_text())
        else:
            team.append('N/A')
    #team = [n.find('span', {'class':'visible-sm-inline'}).get_text() for n in team_tag]
    return team

def get_games(url):
    #pagina = get(url)
    #ler_pg = bs(pagina.text, 'html.parser')
    tabela = ler_pg.find('table', {'class':'StatsGridResults table responsive fixed-cols-2 table-left-cols-2 table-expand table-striped table-hover table-noborder table-centered table-condensed'})
    games_tag = tabela.find_all('td', {'align':'right'})
    games = [n.get_text()[0:] for n in games_tag[0::4]]
    return games

def get_index(url):
    #pagina = get(url)
    #ler_pg = bs(pagina.text, 'html.parser')
    tabela = ler_pg.find('table', {'class':'StatsGridResults table responsive fixed-cols-2 table-left-cols-2 table-expand table-striped table-hover table-noborder table-centered table-condensed'})
    games_tag = tabela.find_all('td', {'align':'right'})
    index_r = [n.get_text()[0:] for n in games_tag[1::4]]
    return index_r

def get_aver(url):
    #pagina = get(url)
    #ler_pg = bs(pagina.text, 'html.parser')
    tabela = ler_pg.find('table', {'class':'StatsGridResults table responsive fixed-cols-2 table-left-cols-2 table-expand table-striped table-hover table-noborder table-centered table-condensed'})
    games_tag = tabela.find_all('td', {'align':'right'})
    aver = [n.get_text()[0:] for n in games_tag[2::4]]
    return aver

def get_per(url):
    #pagina = get(url)
    #ler_pg = bs(pagina.text, 'html.parser')
    tabela = ler_pg.find('table', {'class':'StatsGridResults table responsive fixed-cols-2 table-left-cols-2 table-expand table-striped table-hover table-noborder table-centered table-condensed'})
    games_tag = tabela.find_all('td', {'align':'right'})
    per_40 = [n.get_text()[0:] for n in games_tag[3::4]]
    return per_40

def get_season(url):
    season = ler_pg.find('select', {'name':'ctl00$ctl00$ctl00$ctl00$maincontainer$maincontent$contentpane$ctl01$ddlSeasonSingle'}).find('option', {'selected':'selected'}).get_text()
    return season

In [2]:
base_url = 'https://www.euroleague.net/'
stats = f'{base_url}main/statistics'

stats_get = get(stats)
stats_page = bs(stats_get.text, 'html.parser')

In [3]:
seasons = stats_page.find('select', {'name':'ctl00$ctl00$ctl00$ctl00$maincontainer$maincontent$contentpane$ctl01$ddlSeasonSingle'}).find_all('option')

season_dict = {season.get_text() : season.get('value')[11:] for season in seasons}

link_seasons = [f'{stats}?mode=Leaders&entity=Players&seasonmode=Single&seasoncode={season}&cat=Valuation&agg=Accumulated' for season in season_dict.values()]

link_seasons

['https://www.euroleague.net/main/statistics?mode=Leaders&entity=Players&seasonmode=Single&seasoncode=E2020&cat=Valuation&agg=Accumulated',
 'https://www.euroleague.net/main/statistics?mode=Leaders&entity=Players&seasonmode=Single&seasoncode=E2019&cat=Valuation&agg=Accumulated',
 'https://www.euroleague.net/main/statistics?mode=Leaders&entity=Players&seasonmode=Single&seasoncode=E2018&cat=Valuation&agg=Accumulated',
 'https://www.euroleague.net/main/statistics?mode=Leaders&entity=Players&seasonmode=Single&seasoncode=E2017&cat=Valuation&agg=Accumulated',
 'https://www.euroleague.net/main/statistics?mode=Leaders&entity=Players&seasonmode=Single&seasoncode=E2016&cat=Valuation&agg=Accumulated',
 'https://www.euroleague.net/main/statistics?mode=Leaders&entity=Players&seasonmode=Single&seasoncode=E2015&cat=Valuation&agg=Accumulated',
 'https://www.euroleague.net/main/statistics?mode=Leaders&entity=Players&seasonmode=Single&seasoncode=E2014&cat=Valuation&agg=Accumulated',
 'https://www.eurole

In [5]:
lista_rank = []
lista_player = []
lista_team = []
lista_games = []
lista_index = []
lista_aver = []
lista_per = []
lista_season = []

for url in link_seasons:
    season_pg = get(url)
    season_page = bs(season_pg.text, 'html.parser')
    if season_page.find('span', {'class': 'wp-pager-pages'}):
        last_page = int(get_last_page(url))
        last_url = ['{}&page={}'.format(url, n) for n in range(1, last_page+1)]
        for urls in last_url:
            pagina = get(urls)
            ler_pg = bs(pagina.text, 'html.parser')
            lista_rank += get_rank(urls)
            lista_player += get_player(urls)
            lista_team += get_team(urls)
            lista_games += get_games(urls)
            lista_index += get_index(urls)
            lista_aver += get_aver(urls)
            lista_per += get_per(urls)
            print(urls)
            lista_season += [get_season(urls)] * len(get_rank(urls))
        #print(lista_rank, lista_player, lista_team, lista_games, lista_index, lista_aver, lista_per)
print('ACABOU!')

https://www.euroleague.net/main/statistics?mode=Leaders&entity=Players&seasonmode=Single&seasoncode=E2020&cat=Valuation&agg=Accumulated&page=1
https://www.euroleague.net/main/statistics?mode=Leaders&entity=Players&seasonmode=Single&seasoncode=E2020&cat=Valuation&agg=Accumulated&page=2
https://www.euroleague.net/main/statistics?mode=Leaders&entity=Players&seasonmode=Single&seasoncode=E2020&cat=Valuation&agg=Accumulated&page=3
https://www.euroleague.net/main/statistics?mode=Leaders&entity=Players&seasonmode=Single&seasoncode=E2020&cat=Valuation&agg=Accumulated&page=4
https://www.euroleague.net/main/statistics?mode=Leaders&entity=Players&seasonmode=Single&seasoncode=E2020&cat=Valuation&agg=Accumulated&page=5
https://www.euroleague.net/main/statistics?mode=Leaders&entity=Players&seasonmode=Single&seasoncode=E2020&cat=Valuation&agg=Accumulated&page=6
https://www.euroleague.net/main/statistics?mode=Leaders&entity=Players&seasonmode=Single&seasoncode=E2019&cat=Valuation&agg=Accumulated&page=1

In [8]:
'''lista_rank = []
lista_player = []
lista_team = []
lista_games = []
lista_index = []
lista_aver = []
lista_per = []'''

listao = zip(lista_season, lista_rank, lista_player, lista_team, lista_games, lista_index, lista_aver, lista_per)
for l in listao:
    print(l)

('2020-21', '1', 'TAVARES, WALTER', 'Real Madrid', '29', '545', '18.79', '28.61')
('2020-21', '2', 'JAMES, MIKE', 'CSKA Moscow', '25', '498', '19.92', '25.46')
('2020-21', '3', 'MIROTIC, NIKOLA', 'FC Barcelona', '22', '470', '21.36', '33.92')
('2020-21', '4', 'REYNOLDS, JALEN', 'FC Bayern Munich', '28', '459', '16.39', '30.34')
('2020-21', '5', 'VESELY, JAN', 'Fenerbahce Beko Istanbul', '25', '446', '17.84', '24.62')
('2020-21', '6', 'HENRY, PIERRIA', 'TD Systems Baskonia Vitoria-Gasteiz', '28', '442', '15.79', '23.67')
('2020-21', '7', 'DE COLO, NANDO', 'Fenerbahce Beko Istanbul', '23', '430', '18.70', '28.63')
('2020-21', '8', 'MICIC, VASILIJE', 'Anadolu Efes Istanbul', '26', '426', '16.38', '21.50')
('2020-21', '9', 'GRIGONIS, MARIUS', 'Zalgiris Kaunas', '28', '417', '14.89', '21.42')
('2020-21', '10', 'POLONARA, ACHILLE', 'TD Systems Baskonia Vitoria-Gasteiz', '27', '412', '15.26', '21.17')
('2020-21', '11', 'HIGGINS, CORY', 'FC Barcelona', '29', '408', '14.07', '21.33')
('2020-21'

In [9]:
stats_dict = {'season':lista_season, 'rank':lista_rank, 'player':lista_player, 'team':lista_team, 'games':lista_games, 'index':lista_index, 'average':lista_aver, 'per_40min':lista_per}

df = pd.DataFrame(stats_dict)

In [10]:
df.to_csv('Players_Euroleague.csv')